In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
topics_url = 'https://www.imdb.com/search/title/?release_date=2021-01-01,2021-12-31'
response = requests.get(topics_url)

In [3]:
response

<Response [200]>

In [4]:
page_content = response.text
page_content[:500]

In [6]:
doc = BeautifulSoup(response.text,'html.parser')

In [7]:
def get_topics_page():
    topic_url = 'https://www.imdb.com/search/title/?release_date=2021-01-01,2021-12-31'
    response = requests.get(topic_url)
    if response.status_code !=200:
        raise Exception(f'Failed to load page{topic_url}')
    doc = BeautifulSoup(response.text,'html.parser')
    return doc

In [8]:
doc = get_topics_page()

In [9]:
def get_movie_titles(doc):
    selection_class = "lister-item-header"
    movie_title_tags = doc.find_all('h3',{'class':selection_class})
    movie_titles=[]
    for tag in movie_title_tags:
        title = tag.find('a').text
        movie_titles.append(title)
        
    return movie_titles

In [10]:
titles = get_movie_titles(doc)
titles

['The Black Phone',
 'Only Murders in the Building',
 'Sing 2',
 'Spider-Man: No Way Home',
 '1883',
 'No Time to Die',
 'Superman & Lois',
 'Last Night in Soho',
 'Hacks',
 'Dune: Part One',
 'Ghostbusters: Afterlife',
 'Mad God',
 'The Book of Boba Fett',
 "You Don't Know Me",
 'WandaVision',
 'Pleasure',
 'Old',
 'Eternals',
 'Invincible',
 'House of Gucci',
 'Resident Evil: Welcome to Raccoon City',
 'Licorice Pizza',
 'Ojing-eo geim',
 'Verdens verste menneske',
 'Free Guy',
 'Encanto',
 'Yellowjackets',
 'Arcane: League of Legends',
 'Loki',
 'The Card Counter',
 'The Matrix Resurrections',
 'Operation Mincemeat',
 'Shang-Chi and the Legend of the Ten Rings',
 "The King's Man",
 'Marcel the Shell with Shoes On',
 "Don't Look Up",
 'Walker',
 'The Outlaws',
 'The Suicide Squad',
 'The French Dispatch',
 'CODA',
 'Nightmare Alley',
 'Dopesick',
 'Joe Pickett',
 'The Last Duel',
 'Physical',
 'The Wheel of Time',
 'Red Notice',
 'Wrath of Man',
 'Venom: Let There Be Carnage']

In [11]:
def get_movie_year(doc):
    year_selector = "lister-item-year text-muted unbold"
    movie_year_tags = doc.find_all('span',{'class':year_selector})
    movie_year_tagss=[]
    for tag in movie_year_tags:
        movie_year_tagss.append(tag.get_text().strip()[1:5])        
    return movie_year_tagss

In [12]:
years = get_movie_year(doc)

In [13]:
years[:5]

['2021', '2021', '2021', '2021', '2021']

In [14]:
def get_movie_url(doc):
    url_selector="lister-item-header"           
    movie_url_tags=doc.find_all('h3',{'class':url_selector})
    movie_url_tagss=[]
    base_url = 'https://www.imdb.com/'
    for tag in movie_url_tags:
        movie_url_tagss.append('https://www.imdb.com/' + tag.find('a')['href'])
    return movie_url_tagss

In [15]:
urls = get_movie_url(doc)
urls[:5]

['https://www.imdb.com//title/tt7144666/',
 'https://www.imdb.com//title/tt12851524/',
 'https://www.imdb.com//title/tt6467266/',
 'https://www.imdb.com//title/tt10872600/',
 'https://www.imdb.com//title/tt13991232/']

In [16]:
def get_movie_duration(doc):
    
    selection_class="runtime"
    movie_duration_tags=doc.find_all('span',{'class':selection_class})
    movie_duration=[]

    for tag in movie_duration_tags:
        duration = tag.text[:-4]
        movie_duration.append(duration)
        
        
    return movie_duration

In [17]:
durations = get_movie_duration(doc)
durations[:5]

['103', '30', '110', '148', '60']

In [20]:
def get_movie_rating(doc):
    rating_selector="inline-block ratings-imdb-rating"            
    movie_rating_tags=doc.find_all('div',{'class':rating_selector})
    movie_rating_tagss=[]
    for tag in movie_rating_tags:
        movie_rating_tagss.append(tag.get_text().strip())
    return movie_rating_tagss

In [21]:
ratings = get_movie_rating(doc)

In [24]:
df = pd.DataFrame(titles)
df['imdb_ratings'] = ratings
df['urls'] = urls
df['years'] = years
df

,0,imdb_ratings,urls,years
0,The Black Phone,7.3,https://www.imdb.com//title/tt7144666/,2021
1,Only Murders in the Building,8.1,https://www.imdb.com//title/tt12851524/,2021
2,Sing 2,7.4,https://www.imdb.com//title/tt6467266/,2021
3,Spider-Man: No Way Home,8.3,https://www.imdb.com//title/tt10872600/,2021
4,1883,8.8,https://www.imdb.com//title/tt13991232/,2021
5,No Time to Die,7.3,https://www.imdb.com//title/tt2382320/,2021
6,Superman & Lois,7.8,https://www.imdb.com//title/tt11192306/,2021
7,Last Night in Soho,7.1,https://www.imdb.com//title/tt9639470/,2021
8,Hacks,8.2,https://www.imdb.com//title/tt11815682/,2021
9,Dune: Part One,8.0,https://www.imdb.com//title/tt1160419/,2021


In [28]:
df.rename(columns={df.columns[0]:"movie_name"},inplace=True)

In [29]:
df

,movie_name,imdb_ratings,urls,years
0,The Black Phone,7.3,https://www.imdb.com//title/tt7144666/,2021
1,Only Murders in the Building,8.1,https://www.imdb.com//title/tt12851524/,2021
2,Sing 2,7.4,https://www.imdb.com//title/tt6467266/,2021
3,Spider-Man: No Way Home,8.3,https://www.imdb.com//title/tt10872600/,2021
4,1883,8.8,https://www.imdb.com//title/tt13991232/,2021
5,No Time to Die,7.3,https://www.imdb.com//title/tt2382320/,2021
6,Superman & Lois,7.8,https://www.imdb.com//title/tt11192306/,2021
7,Last Night in Soho,7.1,https://www.imdb.com//title/tt9639470/,2021
8,Hacks,8.2,https://www.imdb.com//title/tt11815682/,2021
9,Dune: Part One,8.0,https://www.imdb.com//title/tt1160419/,2021
